In [8]:
# Example RD arrays for dataset/gop (user must fill actual numbers)
import pandas as pd
import numpy as np 
ours = pd.read_csv('benchmark_results/csv_for_latex/ours_data.csv')
hevc = pd.read_csv('benchmark_results/csv_for_latex/hevc_data.csv')
h264 = pd.read_csv('benchmark_results/csv_for_latex/h264_data.csv')
bpp_ours = ours['total_bpp'].values
msssim_ours = ours['MS-SSIM'].values
lpips_ours = ours['LPIPS'].values
fid_ours = ours['FID'].values

bpp_hevc = hevc['total_bpp'].values
msssim_hevc = hevc['MS-SSIM'].values
lpips_hevc = hevc['LPIPS'].values
fid_hevc = hevc['FID'].values

lpips_ours_inv = 1 - lpips_ours
lpips_hevc_inv = 1 - lpips_hevc

fid_ours_inv = 1 - np.array(fid_ours) / np.max([np.max(fid_ours), np.max(fid_hevc)])
fid_hevc_inv = 1 - np.array(fid_hevc) / np.max([np.max(fid_ours), np.max(fid_hevc)])


In [9]:
import numpy as np
from scipy import interpolate, integrate

def bd_rate(bpp1, metric1, bpp2, metric2):
    log_bpp1 = np.log(bpp1)
    log_bpp2 = np.log(bpp2)

    # Polynomial fit
    p1 = np.polyfit(metric1, log_bpp1, 3)
    p2 = np.polyfit(metric2, log_bpp2, 3)

    # Integration limits
    min_int = max(min(metric1), min(metric2))
    max_int = min(max(metric1), max(metric2))

    # Integrate polynomials
    int1 = np.polyint(p1)
    int2 = np.polyint(p2)

    avg1 = np.polyval(int1, max_int) - np.polyval(int1, min_int)
    avg2 = np.polyval(int2, max_int) - np.polyval(int2, min_int)
    avg_diff = (avg2 - avg1) / (max_int - min_int)

    return (np.exp(avg_diff) - 1) * 100


In [10]:
bd_ms = bd_rate(bpp_hevc, msssim_hevc, bpp_ours, msssim_ours)
bd_lpips = bd_rate(bpp_hevc, lpips_hevc_inv, bpp_ours, lpips_ours_inv)
bd_fid = bd_rate(bpp_hevc, fid_hevc_inv, bpp_ours, fid_ours_inv)

print("BD-Rate vs HEVC:")
print(f"  MS-SSIM: {bd_ms:.2f}%")
print(f"  LPIPS: {bd_lpips:.2f}%")
print(f"  FID: {bd_fid:.2f}%")


BD-Rate vs HEVC:
  MS-SSIM: 322.70%
  LPIPS: 312.93%
  FID: 157.07%


/tmp/ipykernel_974384/3625611956.py:1: RankWarning: Polyfit may be poorly conditioned
  bd_ms = bd_rate(bpp_hevc, msssim_hevc, bpp_ours, msssim_ours)
/tmp/ipykernel_974384/3625611956.py:2: RankWarning: Polyfit may be poorly conditioned
  bd_lpips = bd_rate(bpp_hevc, lpips_hevc_inv, bpp_ours, lpips_ours_inv)
/tmp/ipykernel_974384/3625611956.py:3: RankWarning: Polyfit may be poorly conditioned
  bd_fid = bd_rate(bpp_hevc, fid_hevc_inv, bpp_ours, fid_ours_inv)


In [11]:
bpp_hevc

array([0.00937975, 0.01132116, 0.04076806, 0.08583611])

In [12]:
bpp_ours

array([0.029   , 0.047403, 0.053355])